In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 1. 데이터 로드
df = pd.read_csv("dataset/winequality-red.csv")

# 2. 초기 모델 학습 (피처 중요도 파악용)
X = df.drop("quality", axis=1).values
y = df["quality"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rf_init = RandomForestRegressor(n_estimators=100, random_state=42)
rf_init.fit(X_train, y_train)

# 초기 성능
y_pred_init = rf_init.predict(X_test)
rmse_init = np.sqrt(mean_squared_error(y_test, y_pred_init))
print(f"=== [1] 초기 모델 RMSE: {rmse_init:.4f} ===\n")

# ---------------------------------------------------------
# 3. 중요도 하위 3개 피처 선정 및 제거
# ---------------------------------------------------------

# 중요도 추출
feature_names = df.drop("quality", axis=1).columns
importances = rf_init.feature_importances_

# 데이터프레임으로 변환
imp_df = pd.DataFrame({"feature": feature_names, "importance": importances})
imp_df = imp_df.sort_values("importance", ascending=False).reset_index(drop=True)

# 하위 3개 피처 확인
bottom_3_features = imp_df["feature"].tail(3).tolist()
print(f"✂️ 제거할 피처 (하위 3개): {bottom_3_features}")

# 데이터셋 복사 및 피처 제거 (df_sel)
df_sel = df.drop(columns=bottom_3_features)

# ---------------------------------------------------------
# 4. 선택된 피처로 모델 재학습
# ---------------------------------------------------------

# 데이터 준비
X_sel = df_sel.drop("quality", axis=1).values
y_sel = df_sel["quality"].values

X_train_sel, X_test_sel, y_train_sel, y_test_sel = train_test_split(
    X_sel, y_sel, test_size=0.2, random_state=42
)

# 전처리
scaler_sel = StandardScaler()
X_train_sel = scaler_sel.fit_transform(X_train_sel)
X_test_sel = scaler_sel.transform(X_test_sel)

# 모델 재학습
rf_sel = RandomForestRegressor(n_estimators=100, random_state=42)
rf_sel.fit(X_train_sel, y_train_sel)

# 재학습 모델 평가
y_pred_sel = rf_sel.predict(X_test_sel)
rmse_sel = np.sqrt(mean_squared_error(y_test_sel, y_pred_sel))
r2_sel = r2_score(y_test_sel, y_pred_sel)

print(f"\n=== [2] 피처 제거 후 모델 성능 ===")
print(f"RMSE: {rmse_sel:.4f}")
print(f"R2 Score: {r2_sel:.4f}")

# 성능 비교 출력
print(f"\n📈 RMSE 변화: {rmse_init:.4f} -> {rmse_sel:.4f}")
if rmse_sel < rmse_init:
    print("✅ 피처 제거 성공! 성능이 향상되었습니다.")
else:
    print("⚠️ 성능이 비슷하거나 떨어졌습니다. 제거된 피처에 정보가 있었을 수 있습니다.")

# 중요도 시각화 (제거 후)
feature_names_sel = df_sel.drop("quality", axis=1).columns
importances_sel = rf_sel.feature_importances_
imp_df_sel = pd.DataFrame({"feature": feature_names_sel, "importance": importances_sel})
imp_df_sel = imp_df_sel.sort_values("importance", ascending=False)

plt.figure(figsize=(10,5))
sns.barplot(x="importance", y="feature", data=imp_df_sel, palette="viridis")
plt.title("Feature Importance (After Removing Bottom 3)")
plt.tight_layout()
plt.show()